[ 0  1  1  2  2  3  4  5  6  7  7  8  9  9 10 11  8 12 12 12 12 13 13 14
 14 14 14 15  8  8 16 16 17 17 17 17 18 19 19 19 20 20 20 21 21 22  3  3
  8 23 24 25 26 26 26 26  5 27 27 28 28 28 29 30 30 31 31 32 32 33 34 35
 36 36 36 37 37 35 35 36 36 38 38 38 38 39 40 40 40 40 41 41 41 41 41 42
 43 43 44  0]
